In [3]:
import asyncio
import traceback
import sounddevice as sd
from scipy.io.wavfile import write
import base64

from hume import HumeStreamClient
from hume.models.config import BurstConfig, ProsodyConfig

# Record audio
samplerate = 44100  # Hertz
duration = 4  # seconds
filename = 'output.wav'
print(f"Recording for {duration} seconds...")
myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate, channels=1, blocking=True)
print("Recording complete. Saving the audio as output.wav")
write(filename, samplerate, myrecording)

# Function to encode audio (base64 encoding)
def encode_audio(filename):
    with open(filename, 'rb') as audio_file:
        return base64.b64encode(audio_file.read())

# Hume API interaction
async def main():
    try:
        client = HumeStreamClient("1Fuo6eVLpIj6ndhmC5VXllArH67eOcaSA0XLX3sHdU2SdEy5")
        burst_config = BurstConfig()
        prosody_config = ProsodyConfig()
        async with client.connect([burst_config, prosody_config]) as socket:
            encoded_audio = encode_audio(filename)
            await socket.reset_stream()
            result = await socket.send_bytes(encoded_audio)
            print("Received response from Hume")
            return result  # return the result
    except Exception:
        print(traceback.format_exc())

# This line is specific to IPython/Jupyter Notebook:
result = await main()  # 'await' can be used directly in a cell in Jupyter Notebook

print(result)


Recording for 4 seconds...
Recording complete. Saving the audio as output.wav
Received response from Hume
{'burst': {'warning': 'No vocal bursts detected.', 'code': 'W0101'}, 'prosody': {'predictions': [{'time': {'begin': 0.4, 'end': 4.0}, 'emotions': [{'name': 'Admiration', 'score': 0.008134536445140839}, {'name': 'Adoration', 'score': 0.01936083287000656}, {'name': 'Aesthetic Appreciation', 'score': 0.0412583090364933}, {'name': 'Amusement', 'score': 0.14034071564674377}, {'name': 'Anger', 'score': 0.008292632177472115}, {'name': 'Anxiety', 'score': 0.014433115720748901}, {'name': 'Awe', 'score': 0.025380918756127357}, {'name': 'Awkwardness', 'score': 0.01653282158076763}, {'name': 'Boredom', 'score': 0.03196878358721733}, {'name': 'Calmness', 'score': 0.07689502835273743}, {'name': 'Concentration', 'score': 0.09168277680873871}, {'name': 'Contemplation', 'score': 0.0437004491686821}, {'name': 'Confusion', 'score': 0.01305337529629469}, {'name': 'Contempt', 'score': 0.012787164188921

In [ ]:

# find the emotion ids from emotion names
# emotion_id = []
# for  i, e in enumerate(emotions):
#     if e['name'] in emotion_names:
#         emotion_id.append(i)
# print(emotion_id)

In [4]:
emotions = result['prosody']['predictions'][0]['emotions']
emotion_names = ['Anger', 'Calmness', 'Embarrassment', 'Excitement', 'Romance', 'Sadness']
emotion_colors = ['red',   'green',     'yellow',        'orange',      'pink',     'blue']
emotion_id = [4, 9, 22, 26, 38, 39]
emotion_values = [emotions[i]['score'] for i in emotion_id]
print(emotion_values)


[0.008292632177472115, 0.07689502835273743, 0.012177638709545135, 0.14934971928596497, 0.019728396087884903, 0.025075571611523628]


In [5]:
emotion_values

[0.008292632177472115,
 0.07689502835273743,
 0.012177638709545135,
 0.14934971928596497,
 0.019728396087884903,
 0.025075571611523628]